In [ ]:
import os
import sys
my_path = "/kaggle/input/mlsecu" # path to utils folder
sys.path.append(my_path)
from mlsecu.anomaly_detection_use_case import *
from mlsecu.data_exploration_utils import *
from mlsecu.data_preparation_utils import *
import pandas as pd

### Loading Dataset

In [ ]:
TRAIN_TRANSACTION_FILE_PATH = os.path.join('/kaggle/input/ieee-fraud-detection','train_transaction.csv')

In [ ]:
transaction_df = pd.read_csv(TRAIN_TRANSACTION_FILE_PATH)

In [ ]:
transaction_df.head()

In [ ]:
non_fraud_df = transaction_df[transaction_df['isFraud'] == 0][:9700]
fraud_df = transaction_df[transaction_df['isFraud'] == 1][:300]

In [ ]:
non_fraud_df.head()

In [ ]:
fraud_df.head()

In [ ]:
reduced_ftransaction_df = pd.concat([non_fraud_df, fraud_df], ignore_index=True)
reduced_ftransaction_df.head()

In [ ]:
column_number = len(reduced_ftransaction_df.columns)
print(f'{column_number=}')
print(5*'-')

In [ ]:
column_names = list(reduced_ftransaction_df.columns)
print("column_names")
for col in column_names:
    print(col,end = " ")

In [ ]:
dataset_length = len(reduced_ftransaction_df)
print(f'{dataset_length=}')